<h1><center> Covid-19 Vaccination Progress: Visualisation and Analysis </center></h1>

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly import subplots
import plotly.figure_factory as ff

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
from matplotlib.collections import PatchCollection
from matplotlib.patches import PathPatch
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib import image

import seaborn as sns

import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../input/covid-vaccination-dataset/vaccinations.csv')
population_df = pd.read_csv('../input/population-2020/population_2020.csv')

df.head()

In [ ]:
population_df.head()

<div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana; line-height: 1.7em;">
<center> &nbsp; The below plot shows the total number of vaccination doses administered per 100 people in the total population. This is counted as a single dose, and may not equal the total number of people vaccinated, depending on the specific dose regime (e.g. people receive multiple doses).</center>
</div>

In [ ]:
cols = ['location', 'total_vaccinations', 'iso_code', 'daily_vaccinations','total_vaccinations_per_hundred']
vaccines_per_hundred = df[cols].groupby('location').max().sort_values('total_vaccinations', ascending=False).dropna(subset=['total_vaccinations'])
vaccines_per_hundred.reset_index(level=0, inplace=True)
vaccines_per_hundred = vaccines_per_hundred.sort_values('total_vaccinations_per_hundred', ascending=False)
vaccines_per_hundred = vaccines_per_hundred.iloc[0:25]
vaccines_per_hundred.reset_index(level=0, inplace=True)

plt.figure(figsize=(9, 12))
sns.barplot(y=vaccines_per_hundred['location'], x=vaccines_per_hundred['total_vaccinations_per_hundred'], orient='h')
plt.title('COVID-19 vaccine doses administered per 100 people - Top 25 Nations', fontdict = { 'fontsize': 15})
plt.ylabel('Number of vaccinated people per hundred')
plt.xlabel('Countries')

<div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana; line-height: 1.7em;">
<center> &nbsp; 
The below plot shows the total number of people who received all doses prescribed by the vaccination protocol. This data is only available for countries which report the breakdown of doses administered by first and second doses.</center>
</div>

In [ ]:
cols = ['location', 'total_vaccinations', 'iso_code', 'daily_vaccinations', 'people_vaccinated','people_fully_vaccinated']
people_fully_vaccinated = df[cols].groupby('location').max().sort_values('total_vaccinations', ascending=False).dropna(subset=['total_vaccinations'])
people_fully_vaccinated.reset_index(level=0, inplace=True)
drop_rows = ['World', 'High income', 'North America', 'South America', 'Europe', 'European Union', 'Asia', 'Africa', 'Upper middle income', 'Lower middle income']

people_fully_vaccinated = people_fully_vaccinated[~people_fully_vaccinated['location'].isin(drop_rows)]


people_fully_vaccinated = people_fully_vaccinated.sort_values('people_fully_vaccinated', ascending=False)
people_fully_vaccinated = people_fully_vaccinated.iloc[0:15]
people_fully_vaccinated.reset_index(level=0, inplace=True)

plt.figure(figsize=(9, 12))
sns.barplot(y=people_fully_vaccinated['location'], x=people_fully_vaccinated['people_fully_vaccinated'], orient='h')

plt.title('Number of people fully vaccinated against COVID-19 - Top 15 Nations', fontdict = { 'fontsize': 15})
plt.ylabel('Number of people fully vaccinated')
plt.xlabel('Countries')

<blockquote><p style="font-size:16px; color:#159364; font-family:verdana;">While <code>Gibraltar</code> has the highest number of people per hundred, <code>USA</code> has fully vaccinated more peole in terms of absolute numbers.
When we track the progress of the vaccination drives in different countries, we have to factor in the population of the nations for a better picture.
Our next step would be to track progress relative to population.
</p></blockquote>



In [ ]:
relative_population = population_df[['entity', 'population']]
relative_population = relative_population.rename(columns = {'entity':'location'})

people_fully_vaccinated_new = pd.merge(people_fully_vaccinated, relative_population, on="location")
people_fully_vaccinated_new['vaccinated_percentage'] = (people_fully_vaccinated_new['people_fully_vaccinated'] / people_fully_vaccinated_new['population'])*100
people_fully_vaccinated_new['unvaccinated_percentage'] = 100 - people_fully_vaccinated_new['vaccinated_percentage']
people_fully_vaccinated_new = people_fully_vaccinated_new.sort_values('vaccinated_percentage', ascending=False)
plot_df = people_fully_vaccinated_new[['location', 'vaccinated_percentage', 'unvaccinated_percentage']]

ax = plot_df.plot(figsize = (9, 12),
    x = 'location',
    kind = 'barh',
    stacked = True,
    title = 'Percentage of People Fully Vaccinated',
    mark_right = True,
    colormap='Paired')

ax.set_xlabel("Percentage")
ax.set_ylabel("Country")

<blockquote><p style="font-size:16px; color:#159364; font-family:verdana;">There is a sharp contrast when absolute number of vaccinated people is compared againts the percentage of people fully vaccinated i.e. when population is factored in.
</p></blockquote>

In [ ]:
relative_population = population_df[['entity', 'population']]
relative_population = relative_population.rename(columns = {'entity':'location'})

people_vaccinated_new = pd.merge(people_fully_vaccinated, relative_population, on="location")
people_vaccinated_new['fully_vaccinated'] = (people_vaccinated_new['people_fully_vaccinated'] / people_vaccinated_new['population'])*100
people_vaccinated_new['partially_vaccinated'] = (people_vaccinated_new['people_vaccinated'] / people_vaccinated_new['population'])*100 - people_vaccinated_new['fully_vaccinated']
people_vaccinated_new['atleast_one_dose'] = people_vaccinated_new['fully_vaccinated'] + people_vaccinated_new['partially_vaccinated']
people_vaccinated_new = people_vaccinated_new.sort_values('atleast_one_dose', ascending=False)
plot_df = people_vaccinated_new[['location', 'fully_vaccinated', 'partially_vaccinated']]

ax = plot_df.plot(figsize = (9, 12),
    x = 'location',
    kind = 'barh',
    stacked = True,
    title = 'Percentage of People Fully and Partially Vaccinated',
    mark_right = True,
    colormap='Paired')

ax.set_xlabel("Percentage")
ax.set_ylabel("Country")

In [ ]:
cols = ['location', 'total_vaccinations', 'iso_code', 'daily_vaccinations','people_fully_vaccinated']
people_fully_vaccinated = df[cols].groupby('location').max().sort_values('total_vaccinations', ascending=False).dropna(subset=['total_vaccinations'])
people_fully_vaccinated.reset_index(level=0, inplace=True)
drop_rows = ['World', 'High income', 'North America', 'South America', 'Europe', 'European Union', 'Asia', 'Africa', 'Upper middle income', 'Lower middle income']

people_fully_vaccinated = people_fully_vaccinated[~people_fully_vaccinated['location'].isin(drop_rows)]

people_fully_vaccinated = people_fully_vaccinated.sort_values('people_fully_vaccinated', ascending=False)
people_fully_vaccinated.reset_index(level=0, inplace=True)

people_fully_vaccinated_total = pd.merge(people_fully_vaccinated, relative_population, on="location")

people_fully_vaccinated_total['vaccinated_percentage'] = (people_fully_vaccinated_total['people_fully_vaccinated'] / people_fully_vaccinated_total['population'])*100
people_fully_vaccinated_total['vaccinated_percentage'] = people_fully_vaccinated_total['vaccinated_percentage'].fillna(0)

In [ ]:
fig = px.choropleth(
    people_fully_vaccinated_total,                          
    locations="iso_code",          
    color="vaccinated_percentage",                    
    hover_name="location",             
    color_continuous_scale='balance',
    projection="natural earth",      
    range_color=[0,100],
    title='<span style="font-size:36px; font-family:Times New Roman">Percentage of People Fully Vaccinated</span>',
)
fig.show() 


<h3>There is a long way to go!</h3>
Hardly any countries are in the "lighter" scale indicating a higher percentage of fully vaccinated population.